In [28]:
import numpy as np
import pandas as pd

In [114]:
lido = (pd
        .read_csv('assets/lido_transactions.csv',
                  parse_dates=['CreateDate'],
                  date_format='%Y-%m-%d %H:%M:%S',
                  dtype={'Store': 'category', 'Gross': 'float64', 'HeadCount': 'int64'},
                  usecols=['CreateDate', 'Gross', 'Store', 'DeleteById', 'HeadCount', 'Tags'])
        .where(lambda x: x['DeleteById'].isna())
        .dropna(how='all')
        .drop(columns=['DeleteById']))

yanzi = (pd
        .read_csv('assets/yanzi_transactions.csv',
                  parse_dates=['CreateDate'],
                  date_format='%Y-%m-%d %H:%M:%S',
                  dtype={'Store': 'category', 'Gross': 'float64', 'HeadCount': 'int64'},
                  usecols=['CreateDate', 'Gross', 'Store', 'DeleteById', 'HeadCount', 'Tags'])
        .where(lambda x: x['DeleteById'].isna())
        .dropna(how='all')
        .drop(columns=['DeleteById']))

transactions = pd.concat([lido, yanzi], ignore_index=True)
transactions

,HeadCount,Gross,Tags,Store,CreateDate
0,1.0,255.0,#DineIn,Erod,2022-03-23 08:54:52
1,1.0,255.0,#DineIn,Erod,2022-03-23 09:00:02
2,3.0,1005.0,#DineIn#SeniorDiscount#VatExempt,Erod,2022-03-23 09:01:29
3,1.0,515.0,TakeOut#Grab,Erod,2022-03-23 09:39:35
4,1.0,1547.0,Delivery#FoodPanda,Erod,2022-03-23 09:44:01
...,...,...,...,...,...
342422,4.0,1718.0,#DineIn#SeniorDiscount#VatExempt,Banawe,2025-04-19 12:43:10
342423,1.0,918.0,#DineIn,Banawe,2025-04-19 12:55:23
342424,2.0,1554.0,#DineIn#SeniorDiscount#VatExempt,Banawe,2025-04-19 13:54:35
342425,2.0,1610.0,#DineIn#SeniorDiscount#VatExempt,Banawe,2025-04-19 14:05:38


In [115]:
# populate transaction type column
transaction_types = ['DineIn', 'TakeOut', 'Delivery', 'Institutional', 'Function']
conditions = [transactions['Tags'].str.contains(ttype, na=False) for ttype in transaction_types]
transactions['TransactionType'] = np.select(conditions, transaction_types, default=None)
transactions['TransactionType'] = transactions['TransactionType'].astype('category')

# populate the third party column
third_parties = ['FoodPanda', 'Ons', 'Grab']
conditions = [transactions['Tags'].str.contains(x, na=False) for x in third_parties]
transactions['ThirdParty'] = np.select(conditions, third_parties, default=None)
transactions['ThirdParty'] = transactions['ThirdParty'].astype('category')

transactions = transactions.drop(columns=['Tags'])
transactions

,HeadCount,Gross,Store,CreateDate,TransactionType,ThirdParty
0,1.0,255.0,Erod,2022-03-23 08:54:52,DineIn,NaN
1,1.0,255.0,Erod,2022-03-23 09:00:02,DineIn,NaN
2,3.0,1005.0,Erod,2022-03-23 09:01:29,DineIn,NaN
3,1.0,515.0,Erod,2022-03-23 09:39:35,TakeOut,Grab
4,1.0,1547.0,Erod,2022-03-23 09:44:01,Delivery,FoodPanda
...,...,...,...,...,...,...
342422,4.0,1718.0,Banawe,2025-04-19 12:43:10,DineIn,NaN
342423,1.0,918.0,Banawe,2025-04-19 12:55:23,DineIn,NaN
342424,2.0,1554.0,Banawe,2025-04-19 13:54:35,DineIn,NaN
342425,2.0,1610.0,Banawe,2025-04-19 14:05:38,DineIn,NaN


In [116]:
def filter_by_column_and_add_new_column_using_another_column_value(condition, new_column_name, value_col):
    transactions.loc[condition, new_column_name] = transactions[value_col]

def filter_by_column_and_add_new_column_with_value(condition, new_column_name, value):
    transactions.loc[condition, new_column_name] = value

def _gross_col(name): return name + 'Gross'


def _hc_col(name): return name + 'HeadCount'


def _tc_col(name): return name + 'TransactionCount'


def _ac_col(name): return name + 'AverageCheque'

for t in transaction_types:
    filter_by_column_and_add_new_column_using_another_column_value(
        transactions['TransactionType'] == t, _gross_col(t), 'Gross')

    filter_by_column_and_add_new_column_using_another_column_value(
        transactions['TransactionType'] == t, _hc_col(t), 'HeadCount')

    filter_by_column_and_add_new_column_with_value(
        transactions['TransactionType'] == t, _tc_col(t), 1)

for tp in third_parties:
    filter_by_column_and_add_new_column_using_another_column_value(
        transactions['ThirdParty'] == tp, _gross_col(tp), 'Gross')

    filter_by_column_and_add_new_column_using_another_column_value(
        transactions['ThirdParty'] == tp, _hc_col(tp), 'HeadCount')
    filter_by_column_and_add_new_column_with_value(
        transactions['ThirdParty'] == tp, _tc_col(tp), 1)

day_parts = ['Breakfast', 'Lunch', 'Merienda', 'Dinner']

conditions = {
    'Breakfast': transactions['CreateDate'].dt.hour.between(7, 10),
    'Lunch': transactions['CreateDate'].dt.hour.between(11, 14),
    'Merienda': transactions['CreateDate'].dt.hour.between(15, 17),
    'Dinner': transactions['CreateDate'].dt.hour.between(18, 24)
}

for d in day_parts:
    filter_by_column_and_add_new_column_using_another_column_value(
        conditions[d], _gross_col(d), 'Gross')
    filter_by_column_and_add_new_column_using_another_column_value(
        conditions[d], _hc_col(d), 'HeadCount')
    filter_by_column_and_add_new_column_with_value(
        conditions[d], _tc_col(d), 1)

transactions.head()

,HeadCount,Gross,Store,CreateDate,TransactionType,ThirdParty,DineInGross,DineInHeadCount,DineInTransactionCount,TakeOutGross,...,BreakfastTransactionCount,LunchGross,LunchHeadCount,LunchTransactionCount,MeriendaGross,MeriendaHeadCount,MeriendaTransactionCount,DinnerGross,DinnerHeadCount,DinnerTransactionCount
0,1.0,255.0,Erod,2022-03-23 08:54:52,DineIn,NaN,255.0,1.0,1.0,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,255.0,Erod,2022-03-23 09:00:02,DineIn,NaN,255.0,1.0,1.0,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,1005.0,Erod,2022-03-23 09:01:29,DineIn,NaN,1005.0,3.0,1.0,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,515.0,Erod,2022-03-23 09:39:35,TakeOut,Grab,NaN,NaN,NaN,515.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1547.0,Erod,2022-03-23 09:44:01,Delivery,FoodPanda,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342427 entries, 0 to 342426
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   HeadCount                      342427 non-null  float64       
 1   Gross                          342427 non-null  float64       
 2   Store                          342427 non-null  object        
 3   CreateDate                     342427 non-null  datetime64[ns]
 4   TransactionType                342427 non-null  category      
 5   ThirdParty                     106821 non-null  category      
 6   DineInGross                    185603 non-null  float64       
 7   DineInHeadCount                185603 non-null  float64       
 8   DineInTransactionCount         185603 non-null  float64       
 9   TakeOutGross                   143522 non-null  float64       
 10  TakeOutHeadCount               143522 non-null  float64       
 11  

In [118]:
grouped = transactions.groupby([transactions['CreateDate'].dt.date, 'Store'], observed=False)

In [119]:
aggregate_columns = {
    'Gross': ['sum', 'count', 'mean'],
    'HeadCount': 'sum',
}

for t in transaction_types:
    aggregate_columns[_gross_col(t)] = 'sum'
    aggregate_columns[_hc_col(t)] = 'sum'
    aggregate_columns[_tc_col(t)] = 'sum'

for tp in third_parties:
    aggregate_columns[_gross_col(tp)] = 'sum'
    aggregate_columns[_hc_col(tp)] = 'sum'
    aggregate_columns[_tc_col(tp)] = 'sum'

for d in day_parts:
    aggregate_columns[_gross_col(d)] = 'sum'
    aggregate_columns[_hc_col(d)] = 'sum'
    aggregate_columns[_tc_col(d)] = 'sum'

In [120]:
agg = grouped.agg(aggregate_columns)

cols_for_averages = transaction_types + third_parties + day_parts

for col in cols_for_averages:
    agg[_ac_col(col)] = agg[_gross_col(col)] / agg[_tc_col(col)]

agg.columns = ['_'.join(col).strip().removesuffix('_').removesuffix('_sum') for col in agg.columns]

agg.columns

Index(['Gross', 'Gross_count', 'Gross_mean', 'HeadCount', 'DineInGross',
       'DineInHeadCount', 'DineInTransactionCount', 'TakeOutGross',
       'TakeOutHeadCount', 'TakeOutTransactionCount', 'DeliveryGross',
       'DeliveryHeadCount', 'DeliveryTransactionCount', 'InstitutionalGross',
       'InstitutionalHeadCount', 'InstitutionalTransactionCount',
       'FunctionGross', 'FunctionHeadCount', 'FunctionTransactionCount',
       'FoodPandaGross', 'FoodPandaHeadCount', 'FoodPandaTransactionCount',
       'OnsGross', 'OnsHeadCount', 'OnsTransactionCount', 'GrabGross',
       'GrabHeadCount', 'GrabTransactionCount', 'BreakfastGross',
       'BreakfastHeadCount', 'BreakfastTransactionCount', 'LunchGross',
       'LunchHeadCount', 'LunchTransactionCount', 'MeriendaGross',
       'MeriendaHeadCount', 'MeriendaTransactionCount', 'DinnerGross',
       'DinnerHeadCount', 'DinnerTransactionCount', 'DineInAverageCheque',
       'TakeOutAverageCheque', 'DeliveryAverageCheque',
       'Instituti

In [121]:
agg = agg.rename(columns={
    'CreateDate': 'Date',
    'Store': 'Store',
    'Gross': 'TotalGross',
    'HeadCount': 'TotalHeadCount',
    'Gross_count': 'TotalTransactionCount',
    'Gross_mean': 'TotalAverageCheque',
})

agg.head()

,,TotalGross,TotalTransactionCount,TotalAverageCheque,TotalHeadCount,DineInGross,DineInHeadCount,DineInTransactionCount,TakeOutGross,TakeOutHeadCount,TakeOutTransactionCount,...,DeliveryAverageCheque,InstitutionalAverageCheque,FunctionAverageCheque,FoodPandaAverageCheque,OnsAverageCheque,GrabAverageCheque,BreakfastAverageCheque,LunchAverageCheque,MeriendaAverageCheque,DinnerAverageCheque
CreateDate,Store,,,,,,,,,,,,,,,,,,,,,
2022-03-23,Erod,31055.0,38,817.236842,58.0,13085.0,34.0,18.0,15498.0,22.0,18.0,...,1236.000000,NaN,NaN,1547.000000,925.000000,987.545455,721.583333,861.384615,NaN,NaN
2022-05-05,Erod,71018.0,71,1000.253521,115.0,29351.0,62.0,25.0,33146.0,44.0,38.0,...,1065.125000,NaN,NaN,906.200000,1287.333333,943.160000,NaN,1060.500000,1040.333333,989.701754
2022-06-05,Erod,100638.0,98,1026.918367,141.0,23502.0,65.0,30.0,60205.0,57.0,56.0,...,1410.916667,NaN,NaN,1234.666667,1417.222222,1075.179487,1145.666667,973.375000,1079.473684,817.000000
2022-06-06,Erod,78481.0,72,1090.013889,117.0,31014.0,72.0,29.0,29503.0,31.0,29.0,...,1283.142857,NaN,NaN,1214.333333,1407.000000,976.526316,932.875000,1243.294118,1022.636364,912.181818
2022-06-07,Erod,113631.0,88,1291.261364,144.0,41883.0,83.0,34.0,57054.0,51.0,45.0,...,1632.666667,NaN,NaN,1671.500000,1601.600000,934.371429,866.210526,1215.685714,1217.388889,2044.437500


In [122]:
columns = ['TotalGross', 'TotalHeadCount', 'TotalTransactionCount', 'TotalAverageCheque',
           'DineInGross', 'DineInHeadCount', 'DineInTransactionCount', 'DineInAverageCheque',
           'TakeOutGross', 'TakeOutHeadCount', 'TakeOutTransactionCount', 'TakeOutAverageCheque',
           'FunctionGross', 'FunctionHeadCount', 'FunctionTransactionCount', 'FunctionAverageCheque',
           'InstitutionalGross', 'InstitutionalHeadCount', 'InstitutionalTransactionCount',
           'InstitutionalAverageCheque',
           'DeliveryGross', 'DeliveryHeadCount', 'DeliveryTransactionCount', 'DeliveryAverageCheque',
           'BreakfastGross', 'BreakfastHeadCount', 'BreakfastTransactionCount', 'BreakfastAverageCheque',
           'LunchGross', 'LunchHeadCount', 'LunchTransactionCount', 'LunchAverageCheque',
           'DinnerGross', 'DinnerHeadCount', 'DinnerTransactionCount', 'DinnerAverageCheque',
           'MeriendaGross', 'MeriendaHeadCount', 'MeriendaTransactionCount', 'MeriendaAverageCheque',
           'FoodPandaGross', 'FoodPandaHeadCount', 'FoodPandaTransactionCount', 'FoodPandaAverageCheque',
           'OnsGross', 'OnsHeadCount', 'OnsTransactionCount', 'OnsAverageCheque',
           'GrabGross', 'GrabHeadCount', 'GrabTransactionCount', 'GrabAverageCheque']

agg = agg[columns]
agg = agg.fillna(0).round(5)
agg.head()

,,TotalGross,TotalHeadCount,TotalTransactionCount,TotalAverageCheque,DineInGross,DineInHeadCount,DineInTransactionCount,DineInAverageCheque,TakeOutGross,TakeOutHeadCount,...,FoodPandaTransactionCount,FoodPandaAverageCheque,OnsGross,OnsHeadCount,OnsTransactionCount,OnsAverageCheque,GrabGross,GrabHeadCount,GrabTransactionCount,GrabAverageCheque
CreateDate,Store,,,,,,,,,,,,,,,,,,,,,
2022-03-23,Erod,31055.0,58.0,38,817.23684,13085.0,34.0,18.0,726.94444,15498.0,22.0,...,1.0,1547.00000,925.0,1.0,1.0,925.00000,10863.0,11.0,11.0,987.54545
2022-05-05,Erod,71018.0,115.0,71,1000.25352,29351.0,62.0,25.0,1174.04000,33146.0,44.0,...,5.0,906.20000,3862.0,3.0,3.0,1287.33333,23579.0,25.0,25.0,943.16000
2022-06-05,Erod,100638.0,141.0,98,1026.91837,23502.0,65.0,30.0,783.40000,60205.0,57.0,...,3.0,1234.66667,12755.0,15.0,9.0,1417.22222,41932.0,39.0,39.0,1075.17949
2022-06-06,Erod,78481.0,117.0,72,1090.01389,31014.0,72.0,29.0,1069.44828,29503.0,31.0,...,9.0,1214.33333,7035.0,5.0,5.0,1407.00000,18554.0,19.0,19.0,976.52632
2022-06-07,Erod,113631.0,144.0,88,1291.26136,41883.0,83.0,34.0,1231.85294,57054.0,51.0,...,4.0,1671.50000,8008.0,6.0,5.0,1601.60000,32703.0,35.0,35.0,934.37143


In [125]:
start = pd.Timestamp('2025-03-01').date()
end = pd.Timestamp('2025-03-15').date()

# agg .to_csv('outputs/daily_sales_report.csv')
agg.loc[start:end].to_csv(f"outputs/daily_sales_report-{start}-{end}.csv")